In [24]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.agents.assistant_agent import AssistantAgent

topic = "Calculus"
subject = "Mathematics"
standard = 11

# 1. Instantiate the agent directly
agent = AssistantAgent()

# 2. Use 'await' directly (works only in Jupyter/IPython)
assistant_response = await agent.run(
    topic=topic,
    subject=subject,
    standard=standard
)

In [25]:
print(assistant_response)

### Lesson Planning Prompt for Calculus Curriculum

#### CONTEXT
Create a 35-50 lesson curriculum for Calculus, tailored to Standard 11 students. The curriculum should be based on the provided topic guide, which includes an introduction to Calculus, resources, and study tips.

#### OBJECTIVES
The objectives of this curriculum are to:

1. Introduce students to the fundamental concepts of Calculus, including limits, derivatives, and integrals.
2. Develop students' problem-solving skills and ability to apply Calculus to real-world problems.
3. Foster a deep understanding of Calculus and its applications in various fields.

#### CONSTRAINTS
The curriculum should be designed to accommodate the following constraints:

1. Difficulty level: The curriculum should be challenging but manageable for Standard 11 students.
2. Lesson sequence: Break the topic into sequential lessons, ensuring a logical flow of concepts.
3. Pace: Ensure the pace suits a Standard 11 student, allowing for adequate time 

In [26]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.prompts.tutoring_prompt import LESSON_PLANNING_PROMPT
from backend.src.logger import logging
from backend.src.utils import get_llm_lite_1
from langchain_core.messages import SystemMessage
from backend.src.exception import CustomException
import sys

llm_lite = get_llm_lite_1()

async def plan_curriculum(instructions) -> dict:
        try:
            logging.info("Generating curriculum plan...")
            
            prompt = LESSON_PLANNING_PROMPT.format(
                topic = "Calculus",
                subject = "Mathematics",
                standard = 11,
                instructions = instructions
            )

            response = await llm_lite.ainvoke([SystemMessage(content=prompt)])
            
            return {"plannings": response.content}

        except Exception as e:
            raise CustomException(e, sys)

In [27]:
plans = await plan_curriculum(instructions = assistant_response)

In [28]:
print(plans["plannings"])

Lesson 1: Introduction to Calculus
Lesson 2: History and Development of Calculus
Lesson 3: Basic Concepts and Terminology
Lesson 4: Limits and Continuity
Lesson 5: Evaluating Limits
Lesson 6: Properties of Limits
Lesson 7: Squeeze Theorem and Infinite Limits
Lesson 8: Introduction to Derivatives
Lesson 9: Definition of a Derivative
Lesson 10: Rules of Differentiation
Lesson 11: Power Rule and Product Rule
Lesson 12: Quotient Rule and Chain Rule
Lesson 13: Implicit Differentiation
Lesson 14: Higher-Order Derivatives
Lesson 15: Applications of Derivatives
Lesson 16: Introduction to Integrals
Lesson 17: Definition of a Definite Integral
Lesson 18: Basic Integration Rules
Lesson 19: Substitution Method
Lesson 20: Integration by Parts
Lesson 21: Integration by Partial Fractions
Lesson 22: Improper Integrals
Lesson 23: Applications of Integrals
Lesson 24: Area Between Curves
Lesson 25: Volume of Solids
Lesson 26: Surface Area of Solids
Lesson 27: Introduction to Multivariable Calculus
Lesson

In [29]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

if project_root not in sys.path:
    sys.path.append(project_root)

from backend.src.prompts.tutoring_prompt import LESSON_CONTENT_PROMPT
from backend.src.utils import get_llm_1
from langchain_core.messages import SystemMessage
from backend.src.logger import logging
from backend.src.exception import CustomException
import asyncio

llm = get_llm_1()

async def generate_parallel_lessons(plans, topic, subject, standard) -> dict:
        try:
            logging.info("Generating lessons with rate limiting...")

            plannings = plans
            topic = topic
            subject = subject
            standard = standard
            
            lesson_list = [line.strip() for line in plannings.split("\n") if line.strip()]

            # --- KEY FIX: SEMAPHORE ---
            # This limits the agent to only 3 concurrent requests at a time.
            # You can increase to 5 if you have a paid/higher tier API plan.
            sem = asyncio.Semaphore(3) 

            async def generate_single_lesson_safe(lesson_title):
                """Generates content for one lesson, respecting the semaphore limit."""
                async with sem:  # Wait for a 'slot' to open before requesting
                    try:
                        # Add a small delay to be extra safe against RPM limits
                        await asyncio.sleep(0.1) 
                        
                        p = LESSON_CONTENT_PROMPT.format(
                            lesson_title=lesson_title,
                            topic=topic,
                            subject=subject,
                            standard=standard
                        )
                        res = await llm.ainvoke([SystemMessage(content=p)])
                        logging.info(f"Generated: {lesson_title[:30]}...")
                        return lesson_title, res.content
                    
                    except Exception as e:
                        # Log the specific error (likely 429)
                        logging.error(f"Failed to generate {lesson_title}: {str(e)}")
                        # Retry logic could be added here, but for now we return error html
                        return lesson_title, f"<p>Error generating content: {str(e)}</p>"

            # Create tasks
            tasks = [generate_single_lesson_safe(lesson) for lesson in lesson_list]

            # Run tasks (They will now respect the limit of 3 at a time)
            results = await asyncio.gather(*tasks)

            lessons_dict = {title: content for title, content in results}

            logging.info(f"Generated {len(lessons_dict)} lessons successfully.")

            return {"lessons": lessons_dict}

        except Exception as e:
            raise CustomException(e, sys)

In [31]:
lessons = await generate_parallel_lessons(
    plans = plans["plannings"],
    topic = "Calculus",
    subject = "Mathematics",
    standard = 11
)

In [32]:
for title, content in lessons["lessons"].items():

    print(title, "\n")
    print(content)
    print("\n")
    print("-" * 100)

Lesson 1: Introduction to Calculus 

### Lesson 1: Introduction to Calculus

### <h4>What is Calculus?</h4>

Calculus is a branch of mathematics that deals with the study of continuous change, particularly in the context of functions and limits. It is a fundamental subject that has numerous applications in various fields, including physics, engineering, economics, and computer science. Calculus is divided into two main branches: Differential Calculus and Integral Calculus.

### <h4>History of Calculus</h4>

The development of calculus is attributed to several mathematicians, including Sir Isaac Newton and German mathematician Gottfried Wilhelm Leibniz. Newton developed the method of "fluxions," which is equivalent to the modern method of derivatives. Leibniz, on the other hand, developed the notation and formalism that is used today in calculus. The development of calculus was a gradual process that spanned several centuries, with contributions from many mathematicians.

### <h4>Key Co